### Aruco sensor processing

Experimentation for the development of an Aruco tag based sensor processing system. 

In [1]:
import sys
sys.path.append("..")
from settings import Config
import numpy as np
import cv2
import torch
import pprint

from sensorprocessing import sp_aruco

# Move data to GPU (if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
from sensorprocessing.aruco.utils import aruco_display

arucoDict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_5X5_100)
arucoParams = cv2.aruco.DetectorParameters()

In [3]:
# Specify the experiment
experiment = "sensorprocessing_aruco"
run = "aruco_128" 
exp = Config().get_experiment(experiment, run)
pprint.pprint(exp)
sp = sp_aruco.ArucoSensorProcessing(exp, device)

Loading pointer config file: /home/lboloni/.config/BerryPicker/mainsettings.yaml
Loading machine-specific config file: /home/lboloni/Insync/lotzi.boloni@gmail.com/Google Drive/LotziStudy/Code/PackageTracking/BerryPicker/settings/settings-tredy2.yaml
No system dependent experiment file
 /home/lboloni/Insync/lotzi.boloni@gmail.com/Google Drive/LotziStudy/Code/PackageTracking/BerryPicker/settings/experiment-config/Tredy2/sensorprocessing_aruco/aruco_128_sysdep.yaml,
 that is ok, proceeding.
Configuration for experiment: sensorprocessing_aruco/aruco_128 successfully loaded
{'MARKER_COUNT': 10,
 'data_dir': PosixPath('/home/lboloni/Documents/Hackingwork/__Temporary/BerryPicker-experiments/sensorprocessing_aruco/aruco_128'),
 'exp_run_sys_indep_file': PosixPath('/home/lboloni/Documents/Hackingwork/_Checkouts/BerryPicker/BerryPicker/src/experiment_configs/sensorprocessing_aruco/aruco_128.yaml'),
 'group_name': 'sensorprocessing_aruco',
 'latent_size': 128,
 'run_name': 'aruco_128',
 'validati

In [4]:
camera_number = Config().values["robot"]["active_camera_list"][0]
print(camera_number)
#camera_number = 2
cap = cv2.VideoCapture(camera_number)

while(True):
    ret, frame = cap.read()
    if not ret:
        print("Capture frame returned False, exiting the loop")
        break
    frame = cv2.resize(frame, (256, 256))
    # imgresized = frame

    print(frame)

    #z = sp.process(frame)
    #print(z)

    # insert ARUCO code here

    marker_corners, marker_ids, rejected_candidates = cv2.aruco.detectMarkers(
        frame, arucoDict, parameters=arucoParams)
    
    detected_markers = aruco_display(marker_corners, marker_ids, rejected_candidates, frame)

    # end of ARUCO code

    # cv2.imshow('Press q to exit', frame)
    cv2.imshow('Press q to exit', detected_markers)
    ret = cv2.waitKey(1)
    if ret != -1:
        ret2 = ret & 0xFF
        if ret2 == ord('q'):
            break
# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

2
[[[ 67  88  90]
  [ 58  92  95]
  [ 55  91  95]
  ...
  [114 132 129]
  [110 132 130]
  [109 134 136]]

 [[ 65  92  91]
  [ 58  92  94]
  [ 63  89  95]
  ...
  [114 132 129]
  [111 132 130]
  [113 133 131]]

 [[ 56  94  98]
  [ 54  94  97]
  [ 57  93  96]
  ...
  [113 131 128]
  [108 129 125]
  [114 137 133]]

 ...

 [[ 15  20  12]
  [  5  13   6]
  [  1   5   2]
  ...
  [  0  11  28]
  [  0  13  32]
  [  0  15  37]]

 [[  2  17  13]
  [  0  12   5]
  [  0   5   0]
  ...
  [  0  10  23]
  [  0  11  25]
  [  0  18  32]]

 [[  7  15  10]
  [  3   7   4]
  [  0   4   1]
  ...
  [  0   4  16]
  [  0  11  24]
  [  0  12  26]]]
[[[ 58  88  96]
  [ 62  90  94]
  [ 64  93  96]
  ...
  [120 134 130]
  [115 133 129]
  [105 136 135]]

 [[ 45  95  96]
  [ 55  87  94]
  [ 61  88  97]
  ...
  [118 134 129]
  [111 130 128]
  [108 136 132]]

 [[ 58  93  95]
  [ 56  90  95]
  [ 58  96 101]
  ...
  [115 129 127]
  [114 133 131]
  [111 132 129]]

 ...

 [[ 11  14   5]
  [  3  12   6]
  [  0   4   1]
  

In [5]:
print(marker_corners)

()


In [6]:
marker_ids

In [7]:
frame.shape

(256, 256, 3)

This code creates the encoding for the Aruco markers because 

In [8]:
MARKER_COUNT = 10
XMAX = float(frame.shape[1])
YMAX = float(frame.shape[0])
NORMALIZER = np.tile([XMAX, YMAX], 4)

z = np.ones(MARKER_COUNT * (8 + 1)) * -1.0
for id, corners in zip(marker_ids, marker_corners):
    detection = corners[0].flatten() / NORMALIZER
    idn = id[0]
    z[idn * (8+1):(idn+1) * (8+1)-1] = detection


TypeError: 'NoneType' object is not iterable

In [ ]:
z[0:9]